In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats 
from statsmodels.sandbox.stats.runs import runstest_1samp # para la prueba Runs 
from statsmodels.formula.api import ols #modelo lineal del ANOVA
import statsmodels.api as sm #para generar la tabla anova
from tabulate import tabulate # para tabular los dataframe
from statsmodels.stats.multicomp import pairwise_tukeyhsd# pruba de comparaciones multiples 
from scipy.stats import norm
from statsmodels.graphics.tsaplots import plot_acf

In [4]:

def test_normalityKS(data, variable): # Pruaba de Normalidad Kolmogorov-Smirnof 
    """
    data: arreglo de datos a evaluar la normalidad
    variable: string con el nombre de la variable 
    """  
    print(f"\n Análisis de normalidad por Kolmogorov-Smirnov para '{variable}'")

    # Kolmogorov-Smirnov (KS) test
    ks_stat, ks_p = stats.kstest(data, 'norm', args=(np.mean(data), np.std(data)))
    print(f" Estadístico = {ks_stat:.4f}, p-valor = {ks_p:.4f}")

def test_normalitySW(data, variable): # Prueba de Normalizas Shapiro-Wilks 
    """
    data: arreglo de datos a evaluar la normalidad
    variable: string con el nombre de la variable 
    """
    print(f"\n Análisis de normalidad por Shapiro-Wilk para '{variable}'")
    # Shapiro-Wilk test
    shapiro_stat, shapiro_p = stats.shapiro(data)
    print(f"Estadístico = {shapiro_stat:.4f}, p-valor = {shapiro_p:.4f}")
    
def random_test(residuos):
    """
    Parameters
    ----------
    residuos : Array
        DESCRIPTION: Residuos del ANOVA 

    Returns
    -------
    None.

    """
    _, p_runs = runstest_1samp(residuos, correction=False)

    print(f"Prueba de Runs: p-valor={p_runs}")
    
def test_homogeneityL(var1, var2, name1, name2): # Prueba de levene
    """
    var1 y var2: variables a las que se corroborará homocedasticidad 
    name1 y name2: strings con el nombre de las variables
    """
    print(f"\n Análisis de homocedasticidad entre '{name1}' y '{name2}'")

    # Prueba de Levene (no asume normalidad)
    levene_stat, levene_p = stats.levene(var1, var2)
    print(f"Levene test: Estadístico = {levene_stat:.4f}, p-valor = {levene_p:.4f}")

def t_test_one(data,mu,variable): #Prueba T para una muestra
    """
    data: arreglo de datos a comparar
    mu: media poblacional o valor de referencia 
    variable: string con el nombre de la variable que se está comparando
    """
    print(f"Prueba T para una sola muestra para {variable}")
    t_stat, p_value = stats.ttest_1samp(data, mu)
    print(f"Estadístico = {t_stat:.4f}, valor_p = {p_value:.4f}")
    
def box_cox(data): #transformación depotencia   

    transformed_data, lambda_opt = stats.boxcox(data)
    return transformed_data, lambda_opt

def tukey(respuesta,factor, alfa,n_factor):
    """

    Parameters
    ----------
    respuesta : Array
        DESCRIPTION. Array con los datos de la variable respuesta
    factor : Array
        DESCRIPTION.Array con los niveles del factor 
    alfa : Float
        DESCRIPTION. Valor alfa de comparación 
    n_factor : String
        DESCRIPTION. Nombre del factor

    Returns
    -------
    None.

    """
    
    tukey = pairwise_tukeyhsd(respuesta, factor, alpha=alfa)
    print(f"Prueba Tukey para el factor {n_factor}")
    print(tukey)
    
def kruskal_W(df,Respuesta,Factor):
    """
    
    Parameters
    ----------
    df : Data_Frame
        DESCRIPTION. estructura con los datos del experimento
    Respuesta : String
        DESCRIPTION. nombre de la variable respuesta, key del dataframe
    Factor : String
        DESCRIPTION. nombre del factor, key del dataframe

    Returns
    -------
    None.

    """
    grupos_B = [df[Respuesta][df[Factor] == nivel] for nivel in df[Factor].unique()]
    stat_B, p_B = stats.kruskal(*grupos_B)
    print(f"Kruskal-Wallis para {Factor}: H = {stat_B:.4f}, p = {p_B:.4f}")
    
    
def kruskal_interaccion(df,Respuesta,Factor1,Factor2):
    """
    

    Parameters
    ----------
    df : Data_Frame
        DESCRIPTION. estructura con los datos del experimento
    Respuesta : String
        DESCRIPTION. nombre de la variable respuesta, key del dataframe
    Factor1 : String
        DESCRIPTION. nombre del factor1, key del dataframe
    Factor2 : String
        DESCRIPTION.nombre del factor12, key del dataframe

    Returns
    -------
    None.

    """
    
    df['interaccion'] = df[Factor1].astype(str) + "_" + df[Factor2].astype(str) # se genera una columana con las combinaciones entre factores

    grupos_interaccion = [df[Respuesta][df['interaccion'] == nivel] for nivel in df['interaccion'].unique()]
    stat_int, p_int = stats.kruskal(*grupos_interaccion)
    print(f"Kruskal-Wallis para la interacción {Factor1}x{Factor2} p = {p_int:.4f}")
  

### Leer datos estructurados

In [2]:
df = pd.read_excel("3 factores.xlsx")

### Normalidad

In [6]:
#%% Supuesto de normalidad de la variable respuesta
test_normalitySW(df['conductividad'],'Variable Respuesta')


 Análisis de normalidad por Shapiro-Wilk para 'Variable Respuesta'
Estadístico = 0.9516, p-valor = 0.0462


### Transformación

In [7]:
#si no fuera normal haríamos una tranformación
data_t, lambda_opt = box_cox(df['conductividad'])

test_normalitySW(data_t,'Variable Respuesta transformada')


 Análisis de normalidad por Shapiro-Wilk para 'Variable Respuesta transformada'
Estadístico = 0.9617, p-valor = 0.1181


### Guardar en df

In [8]:
df['conductividad_t']=data_t

### Homocedasticidad

##### Nivel 1

In [ ]:
nivel1=df[df['acido']==0]['conductividad_t']
nivel2=df[df['acido']==6]['conductividad_t']
nivel3=df[df['acido']==12]['conductividad_t']
nivel4=df[df['acido']==18]['conductividad_t']

_, levene_p = stats.levene(nivel1,nivel2,nivel3,nivel4)
print(f"Levene test: p-valor = {levene_p:.4f}")

Levene test: p-valor = 0.7518


##### Nivel 2

In [11]:
nivel1=df[df['sal']==0]['conductividad_t']
nivel2=df[df['sal']==10]['conductividad_t']
nivel3=df[df['sal']==20]['conductividad_t']

_, levene_p = stats.levene(nivel1,nivel2,nivel3)
print(f"Levene test: p-valor = {levene_p:.4f}")

Levene test: p-valor = 0.2441


##### Nivel 3

In [12]:
nivel1=df[df['temperatura']==80]['conductividad_t']
nivel2=df[df['temperatura']==100]['conductividad_t']

_, levene_p = stats.levene(nivel1,nivel2)
print(f"Levene test: p-valor = {levene_p:.4f}")

Levene test: p-valor = 0.2417


### Modelo lineal

In [ ]:
# Los ANOVAS son modelos lineales, entonces se debe ajustar el modelo
#se usa una codificación de efectos 
# No puede haber espacios en el nombre de las columnas
# modelo = ols('conductividad_t ~ C(acido)+C(sal)+C(temperatura)+C(acido):C(sal)+C(sal):C(temperatura)+C(acido):C(temperatura)+C(acido):C(sal):C(temperatura)', data=df).fit()

### Ver si son significativos o no

In [17]:
anova_table = sm.stats.anova_lm(modelo, typ=3)
# Mostrar resultados
print(tabulate(anova_table,headers='keys',tablefmt='heavy_grid'))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃                                ┃     sum_sq ┃   df ┃          F ┃       PR(>F) ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Intercept                      ┃ 0.00360628 ┃    1 ┃   1.08187  ┃   0.308646   ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ C(acido)                       ┃ 0.0718978  ┃    3 ┃   7.18967  ┃   0.00131489 ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ C(sal)                         ┃ 0.00425008 ┃    2 ┃   0.637502 ┃   0.537329   ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ C(temperatura)                 ┃ 0.0106797  ┃    1 ┃   3.20386  ┃   0.0860911  ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ C(acido):C(sal)                ┃ 0.0127615  ┃    6 ┃   0.638064 ┃   0.698749   ┃
┣━━━

Mayor p: menos significativo. Borrar acido : sal

Si fuera un efecto simple, no se puede sacar

In [ ]:
# modelo = ols('conductividad_t ~ C(acido)+C(sal)+C(temperatura)+C(sal):C(temperatura)+C(acido):C(temperatura)+C(acido):C(sal):C(temperatura)', data=df).fit()

In [20]:
anova_table = sm.stats.anova_lm(modelo, typ=3)
print(tabulate(anova_table,headers='keys',tablefmt='heavy_grid'))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃                                ┃     sum_sq ┃   df ┃          F ┃       PR(>F) ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Intercept                      ┃ 0.00360628 ┃    1 ┃   1.08187  ┃   0.308646   ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ C(acido)                       ┃ 0.0718978  ┃    3 ┃   7.18967  ┃   0.00131489 ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ C(sal)                         ┃ 0.00425008 ┃    2 ┃   0.637502 ┃   0.537329   ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ C(temperatura)                 ┃ 0.0106797  ┃    1 ┃   3.20386  ┃   0.0860911  ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ C(sal):C(temperatura)          ┃ 0.0195282  ┃    2 ┃   2.92918  ┃   0.0727348  ┃
┣━━━

No se puede sacar el sal solo, le sigue el triple

In [25]:
modelo = ols('conductividad_t ~ C(acido)+C(sal)+C(temperatura)+C(sal):C(temperatura)+C(acido):C(temperatura)', data=df).fit()

In [26]:
anova_table = sm.stats.anova_lm(modelo, typ=3)
print(tabulate(anova_table,headers='keys',tablefmt='heavy_grid'))

┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃                         ┃     sum_sq ┃   df ┃            F ┃        PR(>F) ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ Intercept               ┃ 2.4889e-05 ┃    1 ┃   0.00735044 ┃   0.932152    ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(acido)                ┃ 0.110452   ┃    3 ┃  10.8732     ┃   3.14047e-05 ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(sal)                  ┃ 0.0677898  ┃    2 ┃  10.0101     ┃   0.000349323 ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(temperatura)          ┃ 0.0009965  ┃    1 ┃   0.294295   ┃   0.590824    ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(sal):C(temperatura)   ┃ 0.00533604 ┃    2 ┃   0.787943   ┃   0.462465    ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━

In [ ]:
# modelo = ols('conductividad_t ~ C(acido)+C(sal)+C(temperatura)', data=df).fit()

In [28]:
anova_table = sm.stats.anova_lm(modelo, typ=3)
print(tabulate(anova_table,headers='keys',tablefmt='heavy_grid'))

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃                ┃      sum_sq ┃   df ┃             F ┃        PR(>F) ┃
┣━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ Intercept      ┃ 0.000505414 ┃    1 ┃   0.157135    ┃   0.693864    ┃
┣━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(acido)       ┃ 0.243181    ┃    3 ┃  25.202       ┃   2.09766e-09 ┃
┣━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(sal)         ┃ 0.178328    ┃    2 ┃  27.7214      ┃   2.424e-08   ┃
┣━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(temperatura) ┃ 3.1555e-07  ┃    1 ┃   9.81059e-05 ┃   0.992145    ┃
┣━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ Residual       ┃ 0.131874    ┃   41 ┃ nan           ┃ nan           ┃
┗━━━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━┻━━━━━━━━━━━━━━━┻━━━━━━━━━━━━━━━┛


Ya se puede sacar la temperatura

In [29]:
modelo = ols('conductividad_t ~ C(acido)+C(sal)', data=df).fit()

In [30]:
anova_table = sm.stats.anova_lm(modelo, typ=3)
print(tabulate(anova_table,headers='keys',tablefmt='heavy_grid'))

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃           ┃     sum_sq ┃   df ┃          F ┃        PR(>F) ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ Intercept ┃ 0.00060084 ┃    1 ┃   0.191359 ┃   0.664029    ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(acido)  ┃ 0.243181   ┃    3 ┃  25.8166   ┃   1.25808e-09 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ C(sal)    ┃ 0.178328   ┃    2 ┃  28.3975   ┃   1.58053e-08 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━━━━━┫
┃ Residual  ┃ 0.131874   ┃   42 ┃ nan        ┃ nan           ┃
┗━━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━━━━━┛


Ya todos son significativos

### Supupestos posteriores

##### Extraer residuos

In [31]:
df['Residuos']=modelo.resid

##### Normalidad

In [ ]:
test_normalitySW(df['Residuos'],'Residuos')


 Análisis de normalidad por Shapiro-Wilk para 'Residuos'
Estadístico = 0.9876, p-valor = 0.8878
Prueba T para una sola muestra para Residuos
Estadístico = -0.0000, valor_p = 1.0000


##### Ver la media = 0

In [ ]:
t_test_one(df['Residuos'],0,"Residuos")

Si son aleatorios

In [33]:
random_test(df['Residuos'])

Prueba de Runs: p-valor=0.6299044449375456


### Si cumple el anova

eee